In [ ]:
import numpy as np
import math
import random

# Hyperparameters
embedding_dim = 128
alpha = 0.75
x_max = 100
learning_rate = 0.05
epochs = 5

# Suppose we have a list of playlists, each a list of song_ids.
playlists = [[], []]

# Step 1: Build the co-occurrence dictionary
co_occurrence = {}
for pl in playlists:
    songs = list(set(pl))  # ensure uniqueness if needed
    for i in range(len(songs)):
        for j in range(i + 1, len(songs)):
            s1, s2 = songs[i], songs[j]
            if s1 > s2:
                s1, s2 = s2, s1
            co_occurrence[(s1, s2)] = co_occurrence.get((s1, s2), 0) + 1

# Extract a mapping for song IDs to indices
song_ids = set()
for i, j in co_occurrence.keys():
    song_ids.add(i)
    song_ids.add(j)
song_ids = list(song_ids)
song_to_idx = {song_id: idx for idx, song_id in enumerate(song_ids)}
vocab_size = len(song_ids)

# Convert co_occurrence dict to arrays for training
pairs = []
for (i, j), count in co_occurrence.items():
    pairs.append((song_to_idx[i], song_to_idx[j], count))

# Initialize embeddings and biases
W = np.random.randn(vocab_size, embedding_dim) / math.sqrt(embedding_dim)
W_tilde = np.random.randn(vocab_size, embedding_dim) / math.sqrt(embedding_dim)
b = np.zeros(vocab_size)
b_tilde = np.zeros(vocab_size)


def f(x):
    return (x / x_max) ** alpha if x < x_max else 1


# Training Loop
for epoch in range(epochs):
    random.shuffle(pairs)
    total_loss = 0.0
    for i_idx, j_idx, X_ij in pairs:
        # Compute prediction
        w_i = W[i_idx]
        w_j = W_tilde[j_idx]
        bi = b[i_idx]
        bj = b_tilde[j_idx]

        # current prediction
        pred = np.dot(w_i, w_j) + bi + bj
        logX = math.log(X_ij)
        weight = f(X_ij)
        diff = pred - logX
        loss = weight * (diff**2)
        total_loss += loss

        # gradients
        grad = 2 * weight * diff
        # update W, W_tilde, b, b_tilde
        W[i_idx] -= learning_rate * grad * w_j
        W_tilde[j_idx] -= learning_rate * grad * w_i
        b[i_idx] -= learning_rate * grad
        b_tilde[j_idx] -= learning_rate * grad

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(pairs)}")

# After training, W + W_tilde (or their average) can be used as the final embedding
final_embeddings = (W + W_tilde) / 2.0


# To get recommendations:
# Given a song s, find closest embeddings in cosine similarity space.
def recommend(song_id, top_k=10):
    idx = song_to_idx[song_id]
    v = final_embeddings[idx]
    sims = (
        final_embeddings
        @ v
        / (np.linalg.norm(final_embeddings, axis=1) * np.linalg.norm(v))
    )
    # sort by similarity
    best = np.argsort(-sims)[: top_k + 1]
    return [song_ids[b] for b in best if b != idx][:top_k]


# Evaluate accuracy by checking if the predicted songs appear in test playlists

Epoch 1/5, Loss: 5.938723193290349e-05
Epoch 2/5, Loss: 5.776355248582258e-05
Epoch 3/5, Loss: 5.6186581471393286e-05
Epoch 4/5, Loss: 5.465492306168793e-05
Epoch 5/5, Loss: 5.3167224270568473e-05


In [3]:
recommend("c")

['b', 'a']